<a href="https://colab.research.google.com/github/okhko/Predicting-Employee-Turnover-Complete-Guide-Analysis/blob/master/tgrss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install python-telegram-bot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.6/631.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.9 MB/s eta 0:00:00


In [6]:
import logging
import requests
from telegram import Update
from telegram.ext import Updater, CommandHandler, CallbackContext, JobQueue

In [7]:
# Вставьте сюда ваш токен, который вы получили от BotFather
TOKEN = '6999535584:AAE_lrVznCEPt--U3iAULV3mbTCJXuB5GBo'

CHANNEL_IDS = ['@treugolniklpr', '@LPRalarm']  # Список каналов для мониторинга

KEYWORDS = ['Крым',
            'Крыму',
            'Севастополь',
            'Севастополю',
            'Ракетная опасность Крым',
            'Ракетная опасность Севастополь',
            'Отбой ракетной опасности Севастополь',
            'Отбой ракетной опасности Крым']  # Список ключевых слов для поиска

# Настройка логирования
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

# Функция для обработки команды /start
def start(update: Update, context: CallbackContext) -> None:
    update.message.reply_text('Привет! Тут просиходит собир новостей из выбранных каналов по заданным ключевым словам.')

# Функция для получения новых сообщений из каналов
def fetch_channel_messages(context: CallbackContext) -> None:
    for channel_id in CHANNEL_IDS:
        try:
            response = requests.get(f'https://api.telegram.org/bot{TOKEN}/getUpdates?chat_id={channel_id}')
            data = response.json()
            for result in data['result']:
                message_text = result['message']['text']
                if any(keyword in message_text for keyword in KEYWORDS):
                    context.bot.send_message(chat_id=context.job.context, text=message_text)
        except Exception as e:
            logger.error(f'Ошибка при получении сообщений из {channel_id}: {e}')

def main() -> None:
    # Создаем Updater и диспетчер
    updater = Updater(TOKEN)

    # Получаем диспетчер для регистрации обработчиков
    dispatcher = updater.dispatcher

    # Регистрируем обработчики
    dispatcher.add_handler(CommandHandler("start", start))

    # Создаем задачу для получения сообщений каждые 10 секунд
    job_queue = updater.job_queue
    job_queue.run_repeating(fetch_channel_messages, interval=10, first=0, context=updater.bot)

    # Запуск бота
    updater.start_polling()

    # Ожидание завершения работы
    updater.idle()

if __name__ == '__main__':
    main()

TypeError: Updater.__init__() missing 1 required positional argument: 'update_queue'